## TensorBoard Practice

In [1]:
import tensorflow as tf
import numpy as np

### Logic gate definition

In [2]:
gate_and  = lambda x: x[0] and x[1]
gate_or   = lambda x: x[0] or  x[1]
gate_nand = lambda x: 1 - gate_and(x)
gate_nor  = lambda x: 1 - gate_or(x)
gate_xor  = lambda x: gate_and((gate_or(x), gate_nand(x)))

### Data generation

In [3]:
gate_list = [('NAND', gate_nand), ('NOR', gate_nor), ('XOR', gate_xor)]

x_list = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]]).astype(np.float32)
y_list = {}

for gate_name, gate_body in gate_list:        
    y_list[gate_name] = np.array(map(gate_body, [(x[0], x[1]) for x in x_list])).reshape(4, 1)

### Multi-task neural network design

In [4]:
x      = tf.placeholder(tf.float32, name='x')
y_nand_true = tf.placeholder(tf.float32, name='y_nand')
y_nor_true = tf.placeholder(tf.float32, name='y_nor')
y_xor_true = tf.placeholder(tf.float32, name='y_xor')
    
with tf.name_scope('hidden_layer_1'):
    h1_W      = tf.Variable(tf.random_uniform([2, 5], -1.0, 1.0))
    h1_b      = tf.Variable(tf.zeros([5]))
    h1        = tf.sigmoid(tf.matmul(x, h1_W) + h1_b)
    
with tf.name_scope('hidden_layer_2'):
    h2_W      = tf.Variable(tf.random_uniform([5, 4], -1.0, 1.0))
    h2_b      = tf.Variable(tf.zeros([4]))
    h2        = tf.sigmoid(tf.matmul(h1, h2_W) + h2_b)
    
with tf.name_scope('nand_output'):
    y_nand_W      = tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0))
    y_nand_b      = tf.Variable(tf.zeros([1]))
    y_nand_pred   = tf.sigmoid(tf.matmul(h2, y_nand_W) + y_nand_b)
    
with tf.name_scope('nand_loss'):
    loss_nand   = tf.reduce_mean(-y_nand_true * tf.log(y_nand_pred) - (1-y_nand_true) * tf.log(1-y_nand_pred), name='loss')
    
with tf.name_scope('nor_output'):
    y_nor_W      = tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0), name='weights')
    y_nor_b      = tf.Variable(tf.zeros([1]), name='biases')
    y_nor_pred   = tf.sigmoid(tf.matmul(h2, y_nor_W) + y_nor_b, name='y_pred')
    
with tf.name_scope('nor_loss'):
    loss_nor   = tf.reduce_mean(-y_nor_true * tf.log(y_nor_pred) - (1-y_nor_true) * tf.log(1-y_nor_pred), name='loss')
    
with tf.name_scope('xor_output'):
    y_xor_W      = tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0), name='weights')
    y_xor_b      = tf.Variable(tf.zeros([1]), name='biases')
    y_xor_pred   = tf.sigmoid(tf.matmul(h2, y_xor_W) + y_xor_b, name='y_pred')
    
with tf.name_scope('xor_loss'):
    loss_xor   = tf.reduce_mean(-y_xor_true * tf.log(y_xor_pred) - (1-y_xor_true) * tf.log(1-y_xor_pred), name='loss')
    
with tf.name_scope('joint_loss'):
    loss_total = loss_nand + loss_nor + loss_xor

In [5]:
tf.summary.scalar('nand_loss_trace', loss_nand)
tf.summary.scalar('nor_loss_trace', loss_nor)
tf.summary.scalar('xor_loss_trace', loss_xor)
tf.summary.scalar('joint_loss_trace', loss_total)

<tf.Tensor 'joint_loss_trace:0' shape=() dtype=string>

In [6]:
feed_dict={
    x:      np.concatenate([x_list for _ in range(10)]),
    y_nand_true: np.concatenate([y_list['NAND'] for _ in range(10)]),
    y_nor_true:  np.concatenate([y_list['NOR']  for _ in range(10)]),
    y_xor_true:  np.concatenate([y_list['XOR']  for _ in range(10)])}

In [7]:
with tf.Session() as sess:
    tb = tf.summary.FileWriter('./myboard/logic_gate', sess.graph)
    tb_merged = tf.summary.merge_all()
    
    train_op = tf.train.AdamOptimizer(tf.Variable(0.01)).minimize(loss_total)
    
    sess.run(tf.global_variables_initializer())
    
    for i in xrange(1000):
        summary, _ = sess.run([tb_merged, train_op], feed_dict)
        tb.add_summary(summary, i)
        
    tb.close()